In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
import csv 

In [2]:
df=pd.read_csv('/kaggle/input/medium-articles/medium_articles.csv')
df.head()

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


In [3]:
df.shape

(192368, 6)

In [4]:
df['text'][1000]

'Every teacher and professor had their share of strain brought by the ongoing COVID-19 pandemic.\n\nThrough listening, adaptation, and planning, we made it to the finish line with quite a bit of energy with my students this semester.\n\nThere was one key decision that helped me have a very smooth remote teaching experience: preparing a front-loaded syllabus.\n\nI made that choice consciously, but of course without a prediction about the implications of the then developing pandemic. I aimed to empower students with a broad spectrum of tools early in the semester and use the rest of the time for individual project development.\n\nThis strategy ended up working well for remote teaching.\n\nShaping a classroom through Agency, Adaptation, and Tooling\n\nOver the years, I observed that front-loaded classes and workshops fared much better in many terms. Combined with a longer exploration process, in the end, they turned into a teaching formula for achieving high-yield, high-quality results.\n

In [5]:
df = df[['title', 'text']]
df.shape

(192368, 2)

In [6]:
df = df.drop_duplicates()
df.shape

(190827, 2)

In [7]:
missing_cols, missing_rows = (
    (df.eq("").sum(x))
    .loc[lambda x: x.gt(0)].index
    for x in (0, 1)
)
print(missing_cols, missing_rows)
df.loc[missing_rows, missing_cols]

Index([], dtype='object') Int64Index([], dtype='int64')


""


In [8]:
df.drop(missing_rows, axis=0, inplace=True)
df.shape

(190827, 2)

In [9]:
df.shape

(190827, 2)

In [10]:
count = 0
def generateSummary(blog):
    global count
    count += 1
    if(count <= 41357):
        return 'alreadySummarized'
    print("Summarising blog ", count)
    try:
        sentences=sent_tokenize(blog)
        sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
        stop_words = stopwords.words('english')
        sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
        w2v=Word2Vec(sentence_tokens,vector_size=1,min_count=1,epochs=1000)
        sentence_embeddings=[[w2v.wv.get_vector(word)[0] for word in words] for words in sentence_tokens]
        max_len=max([len(tokens) for tokens in sentence_tokens])
        sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
        similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
        for i,row_embedding in enumerate(sentence_embeddings):
            for j,column_embedding in enumerate(sentence_embeddings):
                similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph, max_iter=600)
        top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
        sentNeeded = round(0.25 * len(sentences)) - 1
        top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:sentNeeded])
        summary = ""
        for sent in sentences:
            if sent in top.keys():
                summary += sent
        return summary
    except:
        return float("NaN")

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
sentences=sent_tokenize(text)
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]

In [9]:
len("hello" + "world")

10

In [11]:
df['text'] = df['text'].str.replace('\n', '<|n|>')

In [12]:
df['text'][27345]

'Terraform ideas<|n|><|n|>Terraform And PagerDuty — Before You Try<|n|><|n|>Worth knowing facts before you start writing PagerDuty infrastructure code<|n|><|n|>Terraform is well known for his vast choice of providers. Among these, we can find PagerDuty integration. If you are having a lot of services, users and teams inside PagerDuty you should consider making such infrastructure managed as IaC.<|n|><|n|>As you spend more and more time writing HCL code you are going to find a few things which you may tolerate or can be completely unacceptable. I do not want to say that terraform PagerDuty provider is bad. Actually, I use it and recommend it. But let’s look at what I discovered in my first attempts.<|n|><|n|>No support for EventRules inside Services<|n|><|n|>Taken from https://community.pagerduty.com/forum/t/automation-of-service-event-rules/1507/2<|n|><|n|>You do not have the ability to deploy EventRules inside Services with terraform. This feature is actually not supported at all in A

In [11]:
str = "sample"
str == "samle"

False

In [20]:
df.loc[df['title'] == "Only 12 ICOs Opening This Week"]

,title,text
31498,Only 12 ICOs Opening This Week,Welcome to the 30th issue of Token Report’s in...


In [22]:
type("nan") != str

False

In [ ]:
import math
filename = "articlesSet2-two.csv"
fields = ['title', 'summary', 'content']
# writing to csv file 
with open(filename, 'a') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 

    def callback(row):
        summary = generateSummary(row['text'])
        if(summary == "alreadySummarized" or type(summary) != str): 
            return
        rows = [row['title'], summary, row['text']]
        csvwriter.writerow(rows)
    df.apply(callback, axis=1)

Summarising blog  41358
Summarising blog  41359
Summarising blog  41360
Summarising blog  41361
Summarising blog  41362
Summarising blog  41363
Summarising blog  41364
Summarising blog  41365
Summarising blog  41366
Summarising blog  41367
Summarising blog  41368
Summarising blog  41369
Summarising blog  41370
Summarising blog  41371
Summarising blog  41372
Summarising blog  41373
Summarising blog  41374
Summarising blog  41375
Summarising blog  41376
Summarising blog  41377
Summarising blog  41378


/opt/conda/lib/python3.7/site-packages/scipy/spatial/distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Summarising blog  41379
Summarising blog  41380
Summarising blog  41381
Summarising blog  41382
Summarising blog  41383
Summarising blog  41384
Summarising blog  41385
Summarising blog  41386
Summarising blog  41387
Summarising blog  41388
Summarising blog  41389
Summarising blog  41390
Summarising blog  41391
Summarising blog  41392
Summarising blog  41393
Summarising blog  41394
Summarising blog  41395
Summarising blog  41396
Summarising blog  41397
Summarising blog  41398
Summarising blog  41399
Summarising blog  41400
Summarising blog  41401
Summarising blog  41402
Summarising blog  41403
Summarising blog  41404
Summarising blog  41405
Summarising blog  41406
Summarising blog  41407
Summarising blog  41408
Summarising blog  41409
Summarising blog  41410
Summarising blog  41411
Summarising blog  41412
Summarising blog  41413
Summarising blog  41414
Summarising blog  41415
Summarising blog  41416
Summarising blog  41417
Summarising blog  41418
Summarising blog  41419
Summarising blog

/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


Summarising blog  41654
Summarising blog  41655
Summarising blog  41656
Summarising blog  41657
Summarising blog  41658
Summarising blog  41659
Summarising blog  41660
Summarising blog  41661
Summarising blog  41662
Summarising blog  41663
Summarising blog  41664
Summarising blog  41665
Summarising blog  41666
Summarising blog  41667
Summarising blog  41668
Summarising blog  41669
Summarising blog  41670
Summarising blog  41671
Summarising blog  41672
Summarising blog  41673
Summarising blog  41674
Summarising blog  41675
Summarising blog  41676
Summarising blog  41677
Summarising blog  41678
Summarising blog  41679
Summarising blog  41680
Summarising blog  41681
Summarising blog  41682
Summarising blog  41683
Summarising blog  41684
Summarising blog  41685
Summarising blog  41686
Summarising blog  41687
Summarising blog  41688
Summarising blog  41689
Summarising blog  41690
Summarising blog  41691
Summarising blog  41692
Summarising blog  41693


/opt/conda/lib/python3.7/site-packages/networkx/algorithms/link_analysis/pagerank_alg.py:495: RuntimeWarning: invalid value encountered in subtract
  err = np.absolute(x - xlast).sum()


Summarising blog  41694
Summarising blog  41695
Summarising blog  41696
Summarising blog  41697
Summarising blog  41698
Summarising blog  41699
Summarising blog  41700
Summarising blog  41701
Summarising blog  41702
Summarising blog  41703
Summarising blog  41704
Summarising blog  41705
Summarising blog  41706
Summarising blog  41707
Summarising blog  41708
Summarising blog  41709
Summarising blog  41710
Summarising blog  41711
Summarising blog  41712
Summarising blog  41713
Summarising blog  41714
Summarising blog  41715
Summarising blog  41716
Summarising blog  41717
Summarising blog  41718
Summarising blog  41719
Summarising blog  41720
Summarising blog  41721
Summarising blog  41722
Summarising blog  41723
Summarising blog  41724
Summarising blog  41725
Summarising blog  41726
Summarising blog  41727
Summarising blog  41728
Summarising blog  41729
Summarising blog  41730
Summarising blog  41731
Summarising blog  41732
Summarising blog  41733
Summarising blog  41734
Summarising blog

For checking 1024 tokens gpt2